In [1]:
import pandas as pd
import pyarrow.parquet as pq

## Endpoint 1

In [17]:
df_steam_games = pq.read_table('../data2/df_steam_games_limpio_final.parquet').to_pandas()

In [18]:
# Elimino las filas donde 'developer' es 'none'
df_steam_games = df_steam_games[df_steam_games.developer != 'none']
df_steam_games = df_steam_games[df_steam_games.developer != '']


In [19]:
# Elimino las filas creadas por el explode
df = df_steam_games.groupby('id').first().reset_index()

In [20]:
# Elimino las columnas no relevantes para la query
df.drop(columns=['genres','app_name','tags','specs'], axis=1, inplace=True)

In [21]:
df.head()

,id,price,developer,anio
0,10,9.99,valve,2000
1,20,4.99,valve,1999
2,30,4.99,valve,2003
3,40,4.99,valve,2001
4,50,4.99,gearbox software,1999


In [22]:
df.to_parquet('df_endpoint1.parquet')

## Endpoint 2

In [30]:
df_reviews = pq.read_table('../data2/df_reviews_limpio_compr.parquet').to_pandas()
df_steam_games = pq.read_table('df_endpoint1.parquet').to_pandas()

In [33]:
# Elijo las columnas y los sentimientos necesarios 
df_endpoint2 = df_reviews[['item_id','sentiment_analysis']]
df_endpoint2 = df_endpoint2[(df_endpoint2['sentiment_analysis'] == 0) | (df_endpoint2['sentiment_analysis'] == 2)]

In [37]:
# Realizo inner join por id de juego
resultado = df_steam_games.merge(df_endpoint2, left_on='id', right_on='item_id', how='inner')

In [40]:
# Elimino columnas duplicadas
resultado.drop('item_id',axis=1, inplace=True)

In [42]:
resultado.shape

(31536, 5)

In [43]:
resultado.to_parquet('df_endpoint2.parquet')

## Endpoint 3

In [24]:
df_reviews = pq.read_table('../data2/df_reviews_limpio_compr.parquet').to_pandas()
df_steam_games = pq.read_table('df_endpoint1.parquet').to_pandas()

In [25]:
# Elijo columnas necesarias
df_reviews = df_reviews[['user_id','recommend','item_id']]
df_steam_games = df_steam_games[['price','id']]

In [27]:
# Hago inner join
resultado = df_reviews.merge(df_steam_games, left_on='item_id', right_on='id', how='inner')

In [28]:
# Elimino columnas duplicadas
resultado.drop('id',axis=1,inplace=True)

In [29]:
# Elimino filas duplicadas
resultado.drop_duplicates(subset=['user_id', 'item_id'], inplace=True)

In [30]:
resultado.reset_index(inplace=True, drop=True)

In [31]:
resultado.head()

,user_id,recommend,item_id,price
0,76561197970982479,true,1250,19.99
1,death-hunter,true,1250,19.99
2,DJKamBer,true,1250,19.99
3,diego9031,true,1250,19.99
4,76561198081962345,true,1250,19.99


In [32]:
resultado.to_parquet('df_endpoint3.parquet')

## Endpoint 4

In [3]:
df_steam_games = pq.read_table('../data2/df_steam_games_limpio_final.parquet').to_pandas()
df_items = pq.read_table('../data2/df_items_limpio_compr.parquet').to_pandas()

In [4]:
# Elijo columnas necesarias
df_steam_games = df_steam_games[['genres','id','anio']]
df_items = df_items[['user_id','item_id','playtime_forever','playtime_2weeks']]

In [5]:
# Inputo nulos de playtime_forever
df_items[( df_items['playtime_2weeks'] != 0.0 ) & ( df_items['playtime_forever'] == 0.0)]

,user_id,item_id,playtime_forever,playtime_2weeks
253453,KioKIzz,434210,0.0,2.0
1093823,ireallydontfuckingcare,359050,0.0,1.0
2763031,camyabish,242700,0.0,29.0


In [6]:
df_items.at[253453, 'playtime_forever'] = df_items.at[253453,'playtime_2weeks']
df_items.at[1093823, 'playtime_forever'] = df_items.at[1093823,'playtime_2weeks']
df_items.at[2763031, 'playtime_forever'] = df_items.at[2763031,'playtime_2weeks']

# Elimino columna 'playtime_2weeks'
df_items.drop('playtime_2weeks',axis=1, inplace=True)

In [7]:
df = df_items.drop_duplicates()

In [8]:
df.reset_index(inplace=True, drop=True)

In [9]:
df.shape

(4859588, 3)

In [10]:
df_steam_games.shape

(1938650, 3)

In [ ]:
#resultado = df_steam_games.merge(df_items, left_on='id', right_on='item_id', how='inner')

## Endpoint 5

In [15]:
df_steam_games = pq.read_table('df_endpoint1.parquet').to_pandas()
df_reviews = pq.read_table('../data2/df_reviews_limpio_compr.parquet').to_pandas()

In [16]:
# Elijo columnas necesarias
df_steam_games = df_steam_games[['id','developer','anio']]
df_reviews = df_reviews[['item_id','recommend','sentiment_analysis']]

In [17]:
# Realizo inner join
resultado = df_steam_games.merge(df_reviews, left_on='id', right_on='item_id', how='inner')

In [18]:
# Elimino columnas duplicadas
resultado.drop('id', axis=1, inplace=True)

In [19]:
# Filtro para recommend true
resultado = resultado[resultado['recommend'] == 'true']

In [20]:
# Filtro para sentimiento positivo
resultado = resultado[resultado['sentiment_analysis'] == 2]

In [21]:
resultado.reset_index(inplace=True, drop=True)

In [23]:
resultado.to_parquet('df_endpoint5.parquet')